In [1]:
import os

from hansel import Crumb
from hansel.operations import joint_value_map, valuesmap_to_dict
import nipype.pipeline.engine as pe
from nipype.algorithms.misc import Gunzip
from nipype.interfaces import spm
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.io import DataSink
from nipype.interfaces.ants import N4BiasFieldCorrection
from nipype.interfaces.base import traits

from neuro_pypes.crumb import DataCrumb
from neuro_pypes.preproc.slicetime_params import STCParametersInterface
from neuro_pypes.interfaces.nilearn import math_img
from neuro_pypes.preproc import get_bounding_box
from neuro_pypes.utils import (
    remove_ext,
    joinstrings,
    selectindex,
    extend_trait_list
)

/root/.pyenv/versions/3.6.7/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [2]:
wf_name = 'spm_rest_preprocessing'

#work_dir = os.path.expanduser(f'~/data/neuro_pypes/{wf_name}/')
work_dir = os.path.expanduser(f'/data/neuro_pypes/{wf_name}/')

#input_dir = os.path.expanduser('~/projects/neuro/multimodal_test_data/raw')
input_dir = os.path.expanduser('/data/raw')

output_dir = os.path.join(work_dir, 'out')
cache_dir = os.path.join(work_dir, 'wd')

data_path = os.path.join(os.path.expanduser(input_dir), '{subject_id}', '{session}', '{image}')
data_crumb = Crumb(data_path, ignore_list=['.*'])
crumb_modalities = {
    'anat': [('image', 'anat_hc.nii.gz')],
    'fmri': [('image', 'rest.nii.gz')]
}

anat_voxel_sizes = [1, 1, 1]

fmri_smoothing_kernel_fwhm = 8

In [3]:
wf = pe.Workflow(name=wf_name, base_dir=work_dir)

# ------------------------------------------------------------------------------------------------
# DATA INPUT AND SINK
# ------------------------------------------------------------------------------------------------
datasource = pe.Node(
    DataCrumb(crumb=data_crumb, templates=crumb_modalities, raise_on_empty=False),
    name='selectfiles'
)

datasink = pe.Node(
    DataSink(parameterization=False, base_directory=output_dir, ),
    name="datasink"
)
    
# basic file name substitutions for the datasink
undef_args = datasource.interface._infields
substitutions = [(name, "") for name in undef_args]
substitutions.append(("__", "_"))

# datasink.inputs.substitutions = extend_trait_list(datasink.inputs.substitutions, substitutions)

# Infosource - the information source that iterates over crumb values map from the filesystem
infosource = pe.Node(interface=IdentityInterface(fields=undef_args), name="infosrc")
infosource.iterables = list(valuesmap_to_dict(joint_value_map(data_crumb, undef_args)).items())
infosource.synchronize = True

# connect the input_wf to the datasink
joinpath = pe.Node(joinstrings(len(undef_args)), name='joinpath')

# Connect the infosrc node to the datasink
input_joins = [(name, 'arg{}'.format(arg_no + 1)) for arg_no, name in enumerate(undef_args)]

wf.connect([
    (infosource, datasource, [(field, field) for field in undef_args]),
    (datasource, joinpath, input_joins),
    (joinpath, datasink, [("out", "container")]),
])

In [4]:
# ------------------------------------------------------------------------------------------------
# ANAT
# ------------------------------------------------------------------------------------------------

# T1 preprocessing nodes

# ANTs N4 Bias field correction
# n4 = N4BiasFieldCorrection()
# n4.inputs.dimension = 3
# n4.inputs.bspline_fitting_distance = 300
# n4.inputs.shrink_factor = 3
# n4.inputs.n_iterations = [50, 50, 30, 20]
# n4.inputs.convergence_threshold = 1e-6
# n4.inputs.save_bias = True
# n4.inputs.input_image = traits.Undefined
# biascor = pe.Node(n4, name="bias_correction")

gunzip_anat = pe.Node(Gunzip(), name="gunzip_anat")

# SPM New Segment
spm_info = spm.Info()
priors_path = os.path.join(spm_info.path(), 'tpm', 'TPM.nii')
segment = spm.NewSegment()
tissue1 = ((priors_path, 1), 1, (True,  True),   (True,  True))
tissue2 = ((priors_path, 2), 1, (True,  True),   (True,  True))
tissue3 = ((priors_path, 3), 2, (True,  True),   (True,  True))
tissue4 = ((priors_path, 4), 3, (True,  True),   (True,  True))
tissue5 = ((priors_path, 5), 4, (True,  False),  (False, False))
tissue6 = ((priors_path, 6), 2, (False, False),  (False, False))
segment.inputs.tissues = [tissue1, tissue2, tissue3, tissue4, tissue5, tissue6]
segment.inputs.channel_info = (0.0001, 60, (True, True))
segment.inputs.write_deformation_fields = [True, True]
segment.inputs.channel_files = traits.Undefined
segment = pe.Node(segment, name="new_segment")

# Apply deformations
normalize_anat = spm.Normalize12(jobtype='write')
normalize_anat.inputs.write_voxel_sizes = anat_voxel_sizes
normalize_anat.inputs.deformation_file = traits.Undefined
normalize_anat.inputs.image_to_align = traits.Undefined
normalize_anat.inputs.write_bounding_box = traits.Undefined
warp_anat = pe.Node(normalize_anat, name="warp_anat")

tpm_bbox = pe.Node(
    Function(function=get_bounding_box, input_names=["in_file"], output_names=["bbox"]),
    name="tpm_bbox"
)
tpm_bbox.inputs.in_file = priors_path

# calculate brain mask from tissue maps
tissues = pe.Node(
    IdentityInterface(fields=["gm", "wm", "csf"], mandatory_inputs=True),
    name="tissues"
)
brain_mask = pe.Node(
    Function(
        function=math_img, 
        input_names=["formula", "out_file", "gm", "wm", "csf"], 
        output_names=["out_file"],
        imports=['from neuro_pypes.interfaces.nilearn import ni2file']),
        name='brain_mask'
)
brain_mask.inputs.out_file = "tissues_brain_mask.nii.gz"
brain_mask.inputs.formula  = "np.abs(gm + wm + csf) > 0"

# Connect the nodes
wf.connect([
    # input to biasfieldcorrection
#     (datasource, biascor, [("anat", "input_image")]),

    # new segment
#     (biascor,      gunzip_anat, [("output_image", "in_file")]),
    (datasource, gunzip_anat, [("anat", "in_file")]),
    (gunzip_anat,  segment, [("out_file", "channel_files")]),

    # Normalize12
    (segment,   warp_anat,  [("forward_deformation_field", "deformation_file")]),
    (segment,   warp_anat,  [("bias_corrected_images",     "apply_to_files")]),
    (tpm_bbox,  warp_anat,  [("bbox",                      "write_bounding_box")]),

    # brain mask from tissues
    (segment, tissues,[
        (("native_class_images", selectindex, 0), "gm"),
        (("native_class_images", selectindex, 1), "wm"),
        (("native_class_images", selectindex, 2), "csf"),
    ]),

    (tissues, brain_mask, [("gm", "gm"), ("wm", "wm"), ("csf", "csf"),]),

    # output
    (warp_anat, datasink,  [("normalized_files",           "anat.@mni")]),
    (segment,   datasink,  [("modulated_class_images",     "anat.tissues.warped"),
                            ("native_class_images",        "anat.tissues.native"),
                            ("transformation_mat",         "anat.transform.@linear"),
                            ("forward_deformation_field",  "anat.transform.@forward"),
                            ("inverse_deformation_field",  "anat.transform.@inverse"),
                            ("bias_corrected_images",      "anat.@biascor")]),
    (brain_mask, datasink, [("out_file",                   "anat.@brain_mask")]),
])


In [7]:
from nipype.interfaces.nipy.preprocess import Trim, ComputeMask

# ------------------------------------------------------------------------------------------------
# FMRI Clean
# ------------------------------------------------------------------------------------------------

# rs-fMRI preprocessing nodes
trim = pe.Node(Trim(), name="trim")

# slice-timing correction
params = pe.Node(STCParametersInterface(in_files=in_file), name='stc_params')
gunzip = pe.Node(Gunzip(), name="gunzip")

stc = spm.SliceTiming()
stc.inputs.in_files = traits.Undefined
stc.inputs.out_prefix = 'stc'
slice_timing = pe.Node(stc, name='slice_timing')

wf.connect([
    # trim
    (datasource, trim, [("rest", "in_file")]),

    # slice time correction
    (trim, params, [("out_file", "in_files")]),
    
    # processing nodes
    (params, gunzip,    [(("in_files",    _pick_first),      "in_file")]),
    (params, stc,       [(("slice_order", _sum_one_to_each), "slice_order"),
                         (("ref_slice",   _sum_one),         "ref_slice"),
                         ("num_slices",                      "num_slices"),
                         ("time_acquisition",                "time_acquisition"),
                         ("time_repetition",                 "time_repetition"),
                         ]),
    (gunzip, stc,       [("out_file",                        "in_files")]),
    
    # output node
    (params, stc_output, [("time_repetition", "time_repetition")]),
    (stc,    stc_output, [("timecorrected_files", "timecorrected_files")]),
])


realign = pe.Node(nipy_motion_correction(), name='realign')

# average
average = pe.Node(
    Function(
        function=mean_img,
        input_names=["in_file"],
        output_names=["out_file"],
        imports=['from neuro_pypes.interfaces.nilearn import ni2file']
    ),
    name='average_epi'
)

mean_gunzip = pe.Node(Gunzip(), name="mean_gunzip")

# co-registration nodes
coreg = pe.Node(spm_coregister(cost_function="mi"), name="coreg_fmri")
brain_sel = pe.Node(Select(index=[0, 1, 2]), name="brain_sel")

# brain mask made with EPI
epi_mask = pe.Node(ComputeMask(), name='epi_mask')

# brain mask made with the merge of the tissue segmentations
tissue_mask = pe.Node(fsl.MultiImageMaths(), name='tissue_mask')
tissue_mask.inputs.op_string = "-add %s -add %s -abs -kernel gauss 4 -dilM -ero -kernel gauss 1 -dilM -bin"
tissue_mask.inputs.out_file = "tissue_brain_mask.nii.gz"

# select tissues
gm_select = pe.Node(Select(index=[0]), name="gm_sel")
wmcsf_select = pe.Node(Select(index=[1, 2]), name="wmcsf_sel")

# noise filter
noise_wf = rest_noise_filter_wf()
wm_select = pe.Node(Select(index=[1]), name="wm_sel")
csf_select = pe.Node(Select(index=[2]), name="csf_sel")

# bandpass filtering
bandpass = pe.Node(
    Function(
        input_names=['files', 'lowpass_freq', 'highpass_freq', 'tr'],
        output_names=['out_files'],
        function=bandpass_filter
    ),
    name='bandpass'
)

# smooth
smooth = pe.Node(
    Function(
        function=smooth_img,
        input_names=["in_file", "fwhm"],
        output_names=["out_file"],
        imports=['from neuro_pypes.interfaces.nilearn import ni2file']
    ),
    name="smooth"
)
smooth.inputs.fwhm = fmri_smoothing_kernel_fwhm
smooth.inputs.out_file = "smooth_std_{}.nii.gz".format(wf_name)

# Connect the nodes
wf.connect([
    # motion correction
    (stc, realign, [("timecorrected_files", "in_file")]),

    # coregistration target
    (realign, average, [("out_file", "in_file")]),
    (average, mean_gunzip, [("out_file", "in_file")]),
    (mean_gunzip, coreg, [("out_file", "target")]),

    # epi brain mask
    (average, epi_mask, [("out_file", "mean_volume")]),

    # coregistration
    (rest_input, coreg, [("anat", "source")]),
    (rest_input, brain_sel, [("tissues", "inlist")]),
    (brain_sel, coreg, [(("out", flatten_list), "apply_to_files")]),

    # tissue brain mask
    (coreg, gm_select, [("coregistered_files", "inlist")]),
    (coreg, wmcsf_select, [("coregistered_files", "inlist")]),
    (gm_select, tissue_mask, [(("out", flatten_list), "in_file")]),
    (wmcsf_select, tissue_mask, [(("out", flatten_list), "operand_files")]),

    # nuisance correction
    (coreg, wm_select, [("coregistered_files", "inlist",)]),
    (coreg, csf_select, [("coregistered_files", "inlist",)]),
    (realign, noise_wf, [("out_file", "rest_noise_input.in_file",)]),
    (tissue_mask, noise_wf, [("out_file", "rest_noise_input.brain_mask")]),
    (wm_select, noise_wf, [(("out", flatten_list), "rest_noise_input.wm_mask")]),
    (csf_select, noise_wf, [(("out", flatten_list), "rest_noise_input.csf_mask")]),

    (realign, noise_wf, [("par_file", "rest_noise_input.motion_params",)]),

    # temporal filtering
    (noise_wf, bandpass, [("rest_noise_output.nuis_corrected", "files")]),
    # (realign,     bandpass,    [("out_file", "files")]),
    (params, bandpass, [("time_repetition", "tr")]),
    (rest_input, bandpass, [
        ("lowpass_freq", "lowpass_freq"),
        ("highpass_freq", "highpass_freq"),
    ]),
    (bandpass, smooth, [("out_files", "in_file")]),

    # output
    (epi_mask, rest_output, [("brain_mask", "epi_brain_mask")]),
    (tissue_mask, rest_output, [("out_file", "tissues_brain_mask")]),
    (realign, rest_output, [
        ("out_file", "motion_corrected"),
        ("par_file", "motion_params"),
    ]),
    (coreg, rest_output, [
        ("coregistered_files", "tissues"),
        ("coregistered_source", "anat"),
    ]),
    (noise_wf, rest_output, [
        ("rest_noise_output.motion_regressors", "motion_regressors"),
        ("rest_noise_output.compcor_regressors", "compcor_regressors"),
        ("rest_noise_output.gsr_regressors", "gsr_regressors"),
        ("rest_noise_output.nuis_corrected", "nuis_corrected"),
        ("rest_noise_output.tsnr_file", "tsnr_file"),
        ("rest_noise_output.art_displacement_files", "art_displacement_files"),
        ("rest_noise_output.art_intensity_files", "art_intensity_files"),
        ("rest_noise_output.art_norm_files", "art_norm_files"),
        ("rest_noise_output.art_outlier_files", "art_outlier_files"),
        ("rest_noise_output.art_plot_files", "art_plot_files"),
        ("rest_noise_output.art_statistic_files", "art_statistic_files"),
    ]),
    (average, rest_output, [("out_file", "avg_epi")]),
    (bandpass, rest_output, [("out_files", "time_filtered")]),
    (smooth, rest_output, [("out_file", "smooth")]),
])
    
# # anat to fMRI registration inputs
# (anat_output, cleanup_wf, [
#     ("tissues_native", "rest_input.tissues"),
#     ("anat_biascorr", "rest_input.anat"),
# ]),

# # clean_up_wf to datasink
# (cleanup_wf, datasink, [
#     ("rest_output.epi_brain_mask", "rest.@epi_brain_mask"),
#     ("rest_output.tissues_brain_mask", "rest.@tissues_brain_mask"),
#     ("rest_output.tissues", "rest.@tissues"),
#     ("rest_output.anat", "rest.@anat"),
#     ("rest_output.motion_regressors", "rest.@motion_regressors"),
#     ("rest_output.compcor_regressors", "rest.@compcor_regressors"),
#     ("rest_output.gsr_regressors", "rest.@gsr_regressors"),
#     ("rest_output.motion_params", "rest.@motion_params"),
#     ("rest_output.motion_corrected", "rest.@motion_corrected"),
#     ("rest_output.nuis_corrected", "rest.@nuis_corrected"),
#     ("rest_output.time_filtered", "rest.@time_filtered"),
#     ("rest_output.smooth", "rest.@smooth"),
#     ("rest_output.avg_epi", "rest.@avg_epi"),
#     ("rest_output.tsnr_file", "rest.@tsnr"),
#     ("rest_output.art_displacement_files", "rest.artifact_stats.@displacement"),
#     ("rest_output.art_intensity_files", "rest.artifact_stats.@art_intensity"),
#     ("rest_output.art_norm_files", "rest.artifact_stats.@art_norm"),
#     ("rest_output.art_outlier_files", "rest.artifact_stats.@art_outlier"),
#     ("rest_output.art_plot_files", "rest.artifact_stats.@art_plot"),
#     ("rest_output.art_statistic_files", "rest.artifact_stats.@art_statistic"),
# ]),
# ])



NameError: name 'in_file' is not defined

In [ ]:
if n_cpus > 1:
    wf.run(plugin=plugin, plugin_args={"n_procs": n_cpus})
else:
    wf.run(plugin=None)